In [1]:
import numpy as np
import pandas as pd
import sys
print sys.path
import sys
sys.path.append('/home/2829902373/.conda/envs/myenv/lib/python2.7/site-packages/xgboost/__init__.pyc')
print sys.path
import xgboost as xgb
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

df_train = pd.read_csv("./train.csv/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("./test.csv/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("./macro.csv", parse_dates=['timestamp'])

['', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python27.zip', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/plat-darwin', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/plat-mac', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/plat-mac/lib-scriptpackages', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/lib-tk', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/lib-old', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/lib-dynload', '/usr/local/Cellar/pyenv/20151124/versions/anaconda-2.4.0/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg', '/usr/local/Cellar/pyenv/20151124/versions/anaconda-2.4.0/lib/python2.7/site-packages/setuptools-18.4-py2.7.egg', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/site-packages', '/usr/local/opt/pyenv/versions/anaconda-2.4.0/lib/python2.7/site-packages/aeosa', '/usr/local/opt/pyenv/versions/anacon

ImportError: No module named xgboost

In [ ]:
# =============================
# =============================
# cleanup
# brings error down a lot by removing extreme price per sqm
#print(df_train.shape)


#Very Strange data
df_train.loc[df_train['id'] == 10092, 'build_year'] = 2007
df_train.loc[df_train['id'] == 10092, 'state'] = 3
df_train.loc[df_train['id'] == 10093, 'build_year'] = 2009



#df_train.loc[df_train.full_sq == 0, 'full_sq'] = 30
#df_train = df_train[df_train.price_doc/df_train.full_sq <= 600000]
#df_train = df_train[df_train.price_doc/df_train.full_sq >= 10000]
#print(df_train.shape)
#print df_train.describe()
# =============================
# =============================

In [2]:
#==============================
y_train =df_train['price_doc'].values
Y = df_train["price_doc"].values
id_test = df_test['id']

#df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
#df_test.drop(['id'], axis=1, inplace=True)
#=============================

In [3]:
# Build df_all = (df_train+df_test).join(df_macro)
num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
print df_all.shape
print df_all.columns
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print df_all.shape
#print df_all.columns
# ==============================
#Clean Strange Data

#full_sq is 0
bad_index = df_all[df_all.full_sq == 0].index
df_all.ix[bad_index, 'full_sq'] = np.NaN

#Life_sq > Full_sq is strange
bad_index = df_all[df_all.life_sq > df_all.full_sq].index
df_all.ix[bad_index, "life_sq"] = np.NaN

#Remove too small Life_Sq data 
bad_index = df_all[df_all.life_sq < 5].index
df_all.ix[bad_index, "life_sq"] = np.NaN

#Remove too small Full_sq data
bad_index = df_all[df_all.full_sq < 5].index
df_all.ix[bad_index, "full_sq"] = np.NaN

#
kitch_is_build_year = [13117]
df_all.ix[kitch_is_build_year, "build_year"] = df_all.ix[kitch_is_build_year, "kitch_sq"]

#Kitch_sq > life_sq is strange
bad_index = df_all[df_all.kitch_sq >= df_all.life_sq].index
df_all.ix[bad_index, "kitch_sq"] = np.NaN

#Kitch_sq == 0 or ==1 is strange
bad_index = df_all[(df_all.kitch_sq == 0).values + (df_all.kitch_sq == 1).values].index
df_all.ix[bad_index, "kitch_sq"] = np.NaN

#
bad_index = df_all[(df_all.full_sq > 210) & (df_all.life_sq / df_all.full_sq < 0.3)].index
df_all.ix[bad_index, "full_sq"] = np.NaN

#Too old house 
bad_index = df_all[df_all.build_year < 1500].index
df_all.ix[bad_index, "build_year"] = np.NaN

#Too new house
bad_index = df_all[df_all.build_year > 2020].index
df_all.ix[bad_index, "build_year"] = np.NaN

#No room 
bad_index = df_all[df_all.num_room == 0].index 
df_all.ix[bad_index, "num_room"] = np.NaN

#No max floor and floor
bad_index = df_all[(df_all.floor == 0).values * (df_all.max_floor == 0).values].index
df_all.ix[bad_index, ["max_floor", "floor"]] = np.NaN

#No floor
bad_index = df_all[df_all.floor == 0].index
df_all.ix[bad_index, "floor"] = np.NaN

#No max floor
bad_index = df_all[df_all.max_floor == 0].index
df_all.ix[bad_index, "max_floor"]=np.NaN

#floor > max_floor 
bad_index = df_all[df_all.floor > df_all.max_floor].index
df_all.ix[bad_index, "max_floor"] = np.NaN

#23584 is strange
bad_index = [23584]
df_all.ix[bad_index, "floor"] = np.NaN

#1 <= state <= 4 
bad_index = df_all[df_all.state == 33].index
df_all.ix[bad_index, "state"] = np.NaN




# ==============================
#Clean Data
#from sklearn.preprocessing import RobustScaler
#rbscaler = RobustScaler()
#df_all["rb_full_sq"] = rbscaler.fit_transform(df_all["full_sq"])


(38133, 290)
Index([u'timestamp', u'full_sq', u'life_sq', u'floor', u'max_floor',
       u'material', u'build_year', u'num_room', u'kitch_sq', u'state',
       ...
       u'cafe_count_5000_price_1500', u'cafe_count_5000_price_2500',
       u'cafe_count_5000_price_4000', u'cafe_count_5000_price_high',
       u'big_church_count_5000', u'church_count_5000', u'mosque_count_5000',
       u'leisure_count_5000', u'sport_count_5000', u'market_count_5000'],
      dtype='object', length=290)
(38133, 390)
Index([u'timestamp', u'full_sq', u'life_sq', u'floor', u'max_floor',
       u'material', u'build_year', u'num_room', u'kitch_sq', u'state',
       ...
       u'provision_retail_space_modern_sqm', u'turnover_catering_per_cap',
       u'theaters_viewers_per_1000_cap',
       u'seats_theather_rfmin_per_100000_cap', u'museum_visitis_per_100_cap',
       u'bandwidth_sports', u'population_reg_sports_share',
       u'students_reg_sports_share', u'apartment_build',
       u'apartment_fund_sqm'],
      d

KeyError: 'id'

In [8]:
# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

0       2011
1       2011
2       2011
3       2011
4       2011
5       2011
6       2011
7       2011
8       2011
9       2011
10      2011
11      2011
12      2011
13      2011
14      2011
15      2011
16      2011
17      2011
18      2011
19      2011
20      2011
21      2011
22      2011
23      2011
24      2011
25      2011
26      2011
27      2011
28      2011
29      2011
        ... 
7632    2016
7633    2016
7634    2016
7635    2016
7636    2016
7637    2016
7638    2016
7639    2016
7640    2016
7641    2016
7642    2016
7643    2016
7644    2016
7645    2016
7646    2016
7647    2016
7648    2016
7649    2016
7650    2016
7651    2016
7652    2016
7653    2016
7654    2016
7655    2016
7656    2016
7657    2016
7658    2016
7659    2016
7660    2016
7661    2016
Name: timestamp, dtype: int64
0       8
1       8
2       8
3       9
4       9
5       9
6       9
7       9
8       9
9       9
10      9
11      9
12      9
13      9
14      9
15      9
16      9
17     

In [ ]:
# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)
df_all["inverse_floor"] = df_all["max_floor"]-df_all["floor"]
df_all["extra_area"] = df_all["full_sq"] - df_all["life_sq"]
df_all["room_size"] = (df_all["life_sq"] - df_all["kitch_sq"])/df_all["num_room"]
df_all["pop_density_raion"] = df_all["raion_popul"]/df_all["area_m"]
df_all["young_proportion"] =df_all["young_all"] / df_all["full_all"]
df_all["work_proportion"] = df_all["work_all"] / df_all["full_all"]
df_all["retire_proportion"] = df_all["ekder_all"] / df_all["full_all"]
df_all["ratio_preschool"] = df_all["children_school"] /df_all["preschool_quota"]
df_all["ratio_school"] = df_all["children_school"] / df_all["school_quota"]
df_all["year_old"] = df_all.timestamp.dt.year-df_all["build_year"]



#df_all["FAR"]=df_all["full_sq"]/df_all["area_m"].astype(float)
#df_all["ratio_life"]=((df_all["full_sq"]-df_all["kitch_sq"])/df_all["life_sq"]).astype(float)
#df_all["sqrt_area"] = np.sqrt(df_all["area_m"])
#df_all["log_full_sq"] = np.log(df_all["full_sq"])
#df_all["log_life_sq"] = np.log(df_all["life_sq"])
#df_all["log_kitch_sq"] = np.log(df_all["kitch_sq"])
#df_all["log_area_m"] = np.log(df_all["area_m"])
#df_all.drop(['full_sq','life_sq','kitch_sq','area_m'],axis=1,inplace=True)
#print df_all.loc[:,["sub_area","full_sq","life_sq","sqrt_area","area_m"]]

#df_all["st_mate"] = df_all["state"]*df_all["material"]

In [ ]:
#clean build_year

#col = "build_year"
#df_all[col].fillna(0,inplace=True)
#df_all["build_year"] = df_all[df_all.build_year.values > 0]
#ulimit = np.percentile(df_all[col].values, 99.995)
#llimit = np.percentile(df_all[col].values, 0.005)
#df_all[col].ix[df_all[col]>ulimit] = ulimit
#df_all[col].ix[df_all[col]<llimit] = llimit


In [ ]:
#df_all["build_year"].describe()
#print df_all.loc[:,["build_count_before_1920","build_count_1921-1945","build_count_1946-1970","build_count_1971-1995","build_count_after_1995"]]
#df_all.loc[:,["timestamp","sub_area","raion_build_count_with_builddate_info","build_count_before_1920","build_count_1921-1945", "build_count_1946-1970","build_count_1971-1995","build_count_after_1995","build_year"]]
#df_all["room_area"] = ((df_all["life_sq"] - df_all["kitch_sq"])/df_all["num_room"]).astype(float)
#df_all.loc[:,["sub_area","raion_popul"]]
#df_all["raion_build_count_with_builddate_info"].describe([0.01,0.99])

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)

factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns


xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
    'lambda': 0.2
    #'alpha':0.001
}

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)


# Uncomment to tune XGB `num_boost_rounds`

cv_result = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
    verbose_eval=True, show_stdv=False)
cv_result[['train-rmse-mean', 'test-rmse-mean']].plot()
num_boost_rounds = len(cv_result)

#num_boost_round = 500

model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_round)

fig, ax = plt.subplots(1, 1, figsize=(8, 16))
xgb.plot_importance(model,height=0.5, ax=ax)
plt.show()

y_pred = model.predict(dtest)
y_pred = np.round(y_pred)
df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})


df_sub.to_csv('sub_530_1.csv', index=False)

In [ ]:
print df_sub